In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
import pickle

In [0]:
movies = pd.read_csv("/content/datanam.csv",encoding="Latin1")
Ratings = pd.read_csv("/content/datarat.csv")
Tags = pd.read_csv("/content/datatag.csv",encoding="Latin1")
Rating,Test=train_test_split(Ratings,test_size=0.2)

Mean = Rating.groupby(by="customer_id",as_index=False)['reviews.rating'].mean()
Rating_avg = pd.merge(Rating,Mean,on='customer_id')
Rating_avg['adg_rating']=Rating_avg['reviews.rating_x']-Rating_avg['reviews.rating_y']
Rating_avg.head()

,product_id,reviews.rating_x,customer_id,reviews.rating_y,adg_rating
0,927,5.0,4361811,4.166667,0.833333
1,11234,5.0,4361811,4.166667,0.833333
2,971,5.0,4361811,4.166667,0.833333
3,1567,4.0,4361811,4.166667,-0.166667
4,14224,5.0,4361811,4.166667,0.833333


In [0]:
Rating_avg.head()


,product_id,reviews.rating_x,customer_id,reviews.rating_y,adg_rating
0,927,5.0,4361811,4.166667,0.833333
1,11234,5.0,4361811,4.166667,0.833333
2,971,5.0,4361811,4.166667,0.833333
3,1567,4.0,4361811,4.166667,-0.166667
4,14224,5.0,4361811,4.166667,0.833333


In [0]:
Rating.head()


,product_id,reviews.rating,customer_id
1157,927,5.0,4361811
3452,7182,5.0,14783707
5890,14224,5.0,6301648
3630,15268,5.0,30646298
5744,14224,5.0,47454014


In [0]:
check = pd.pivot_table(Rating_avg,values='reviews.rating_x',index='customer_id',columns='product_id')
check.head()

product_id,12,60,102,162,432,517,581,815,825,927,971,1142,1324,1362,1527,1543,1567,2517,5617,6123,6178,6179,6182,6192,6271,7182,7241,7343,7519,9172,9713,11234,14224,14251,15123,15268,16273,16892,17231,27182,27881,51681,52762,61723,71521,78129,517162
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
111930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
111932,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
115345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [0]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='customer_id',columns='product_id')
final.head()


product_id,12,60,102,162,432,517,581,815,825,927,971,1142,1324,1362,1527,1543,1567,2517,5617,6123,6178,6179,6182,6192,6271,7182,7241,7343,7519,9172,9713,11234,14224,14251,15123,15268,16273,16892,17231,27182,27881,51681,52762,61723,71521,78129,517162
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.40,-0.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400,NaN,NaN,-0.6,0.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN
111930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.625,NaN,NaN,NaN,NaN,NaN,0.375,NaN,0.375,NaN,NaN,NaN,0.375,-0.625000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375,NaN,NaN,NaN
111932,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.833333,NaN,NaN,0.166667,NaN,0.166667,0.166667,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN
115345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.75,0.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.250,NaN,0.250,NaN,NaN,NaN,1.250,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.750,NaN,NaN,NaN


In [0]:
final_movie = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)
final_user.head()

product_id,12,60,102,162,432,517,581,815,825,927,971,1142,1324,1362,1527,1543,1567,2517,5617,6123,6178,6179,6182,6192,6271,7182,7241,7343,7519,9172,9713,11234,14224,14251,15123,15268,16273,16892,17231,27182,27881,51681,52762,61723,71521,78129,517162
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97309,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,4.000000e-01,-6.000000e-01,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,4.000000e-01,3.552714e-16,3.552714e-16,-6.000000e-01,4.000000e-01,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16,3.552714e-16
110441,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
111930,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,3.750000e-01,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,-6.250000e-01,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,3.750000e-01,8.928571e-02,3.750000e-01,8.928571e-02,8.928571e-02,8.928571e-02,3.750000e-01,-6.250000e-01,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,8.928571e-02,3.750000e-01,8.928571e-02,8.928571e-02,8.928571e-02
111932,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,1.666667e-01,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,-8.333333e-01,2.960595e-16,2.960595e-16,1.666667e-01,2.960595e-16,1.666667e-01,1.666667e-01,2.960595e-16,2.960595e-16,1.666667e-01,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16
115345,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-7.500000e-01,2.500000e-01,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,1.250000e+00,-3.571429e-02,2.500000e-01,-3.571429e-02,-3.571429e-02,-3.571429e-02,1.250000e+00,2.500000e-01,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-3.571429e-02,-2.750000e+00,-3.571429e-02,-3.571429e-02,-3.571429e-02


In [0]:
final_movie.head()

product_id,12,60,102,162,432,517,581,815,825,927,971,1142,1324,1362,1527,1543,1567,2517,5617,6123,6178,6179,6182,6192,6271,7182,7241,7343,7519,9172,9713,11234,14224,14251,15123,15268,16273,16892,17231,27182,27881,51681,52762,61723,71521,78129,517162
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97309,0.298232,0.350232,0.188107,0.026701,0.221561,0.328351,0.079951,-2.0,0.41958,0.400000,-0.600000,-0.049542,0.209716,-0.620714,0.229823,-0.18617,0.379228,0.157534,0.276042,0.137972,0.077908,0.0,0.058447,-0.120068,0.095238,0.101566,0.226935,0.400000,0.22763,-0.102826,-0.600000,0.400000,-0.435026,-0.34127,0.264286,0.443353,0.522619,0.344444,-0.051735,0.357143,-0.330495,0.119038,0.106766,-0.023246,0.044547,0.571429,0.017619
110441,0.298232,0.350232,0.188107,0.026701,0.221561,0.328351,0.079951,-2.0,0.41958,-0.047553,0.272318,-0.049542,0.209716,-0.620714,0.229823,-0.18617,0.379228,0.157534,0.276042,0.000000,0.077908,0.0,0.058447,-0.120068,0.095238,0.101566,0.226935,0.000000,0.22763,-0.102826,0.032879,0.000000,0.000000,-0.34127,0.264286,0.443353,0.522619,0.344444,-0.051735,0.357143,-0.330495,0.119038,0.106766,0.000000,0.044547,0.571429,0.017619
111930,0.298232,0.350232,0.188107,0.026701,0.221561,0.328351,0.079951,-2.0,0.41958,-0.047553,0.375000,-0.049542,0.209716,-0.620714,0.229823,-0.18617,0.379228,0.157534,0.276042,-0.625000,0.077908,0.0,0.058447,-0.120068,0.095238,0.375000,0.226935,0.375000,0.22763,-0.102826,0.032879,0.375000,-0.625000,-0.34127,0.264286,0.443353,0.522619,0.344444,-0.051735,0.357143,-0.330495,0.119038,0.106766,0.375000,0.044547,0.571429,0.017619
111932,0.298232,0.350232,0.188107,0.026701,0.221561,0.328351,0.166667,-2.0,0.41958,-0.047553,0.272318,-0.049542,0.209716,-0.620714,0.229823,-0.18617,0.379228,0.157534,0.276042,0.137972,0.077908,0.0,0.058447,-0.120068,0.095238,0.101566,0.226935,-0.049029,0.22763,-0.102826,0.032879,0.259761,-0.833333,-0.34127,0.264286,0.166667,0.522619,0.166667,0.166667,0.357143,-0.330495,0.166667,0.106766,-0.023246,0.044547,0.571429,0.017619
115345,0.298232,0.350232,0.188107,0.026701,0.221561,0.328351,0.079951,-2.0,0.41958,-0.750000,0.250000,-0.049542,0.209716,-0.620714,0.229823,-0.18617,0.379228,0.157534,0.276042,0.137972,0.077908,0.0,0.058447,-0.120068,0.095238,1.250000,0.226935,0.250000,0.22763,-0.102826,0.032879,1.250000,0.250000,-0.34127,0.264286,0.443353,0.522619,0.344444,-0.051735,0.357143,-0.330495,0.119038,0.106766,-2.750000,0.044547,0.571429,0.017619


In [0]:
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()
filename = 'final_user.pickle'
pickle.dump(b, open(filename, 'wb'))

In [0]:
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()
filename = 'final_movie.pickle'
pickle.dump(similarity_with_movie, open(filename, 'wb'))

In [0]:

def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [0]:
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97309,50772322,25659526,45286795,32038204,21332512,33582678,10876480,14970541,787281,530848,48148197,16189998,52779789,30841819,10504818,14579833,36816790,27053976,32467074,4361811,17559184,49398124,29182640,30059074,42391754,51746033,14908944,49242514,15176382,32133759
110441,53076619,14019850,14414189,14400882,14338649,14324509,14315429,14250726,14225036,14173434,14135088,14054766,14045173,14028763,14007674,13425525,13940487,13895839,13894352,13804717,13659988,13647366,13585462,13577585,13534991,13534413,13532681,13529886,14435268,14458914
111930,27278599,18012236,14414189,14544335,1119536,42133558,2260642,13532681,221245,23616875,34451018,176073,25289305,20745964,11927311,26155827,125528,52352506,12125085,7274926,17105613,21884750,52944305,12405999,22626548,22654450,23826451,39406815,39532206,46917633
111932,1079519,46827382,29671204,52056685,46111284,348783,14019850,11184728,40961802,22178713,52674826,30646298,20307393,34005760,15037519,2575622,42143398,44239442,12061025,52774082,9881235,40085728,34777582,4159021,7822109,52633909,4375234,196208,18653653,43773798
115345,41659434,1578888,4888142,10395219,16277617,319219,2294292,14783707,41217062,52929536,3698606,50828413,17679177,24621469,31418346,43114127,39472629,9409243,49188005,43453006,41143885,24331993,9706523,7431437,31260165,13647366,46609348,30959841,21204337,48959028


In [0]:
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97309,45286795,50772322,14908944,10504818,32467074,25659526,787281,658913,25780394,14970541,29182640,19682713,8225931,24863069,27148028,17161394,17447517,32971923,30841819,50828061,49398124,48148197,20745964,51010048,38360101,2346214,51698598,41861544,51168531,49242514
110441,48424535,31678700,29340349,23516414,43654918,12050162,33072806,43301254,31687702,39412339,792596,23032177,21032808,13585462,133451,10163944,2990247,17447517,22070226,17783165,781699,1067307,2089124,10050486,28710645,39532206,1034122,38195398,50421178,38518519
111930,1119536,46917633,42133558,38367373,32074436,41742791,18179651,20992492,17105613,34612131,51698598,51010048,17161394,34451018,28733547,23775884,39406815,38518519,176073,196208,44157787,7822109,29092404,50828061,16358458,9881235,42194372,40085728,13895839,48805811
111932,1079519,48805811,7822109,29671204,14943157,52056685,44456431,48701722,20992492,44157787,37293313,52774082,3770191,32059236,23775884,34323478,196208,52674826,9097384,12887792,9600366,9881235,37396671,40085728,1195910,11184728,46917633,44239442,14019850,25328707
115345,41659434,4888142,39472629,1578888,16277617,2294292,319219,10395219,24621469,52929536,13647366,17679177,7431437,24331993,9409243,43453006,31418346,31260165,43114127,46609348,48959028,12259473,21204337,9706523,50828413,30959841,41217062,14783707,30987150,3698606


In [0]:

def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.customer_id == user1].merge(
    Rating_avg[Rating_avg.customer_id == user2],
    on = "product_id",
    how = "inner" )
    return common_movies.merge( movies, on = 'product_id' )

In [0]:

a = get_user_similar_movies(99730,99730)
a = a.loc[ : , ['reviews.rating_x_x','reviews.rating_x_y','name']]

a.head()


,reviews.rating_x_x,reviews.rating_x_y,name


In [0]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['customer_id'] == user,'reviews.rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [0]:
score = User_item_score(45880701,14224)
print("score (u,i) is",score)


score (u,i) is 3.101815541770479


In [0]:

Rating_avg = Rating_avg.astype({"product_id": str})
Movie_user = Rating_avg.groupby(by = 'customer_id')['product_id'].apply(lambda x:','.join(x))

In [0]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['customer_id'] == user,'reviews.rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'product_id':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(10)
    Movie_Name = top_5_recommendation.merge(movies, how='inner', on='product_id')
    Movie_Names = Movie_Name.name.values.tolist()
    return Movie_Names

In [0]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : 110441")
print("   ")
for i in predicted_movies:
    print(i)

Enter the user id to whom you want to recommend : 45880701
 
The Recommendations for User Id : 110441
   
Microsoft Surface Pro 4 Type Cover with Fingerprint ID
Midland Weather Alert Radio, 1.0 CT
Sony SRSXB30/BLK XB30 Portable Wireless Speaker with Bluetooth
Sony LBT-GPX555 Mini-System with Bluetooth and NFC
Boytone - 2500W 2.1-Ch. Home Theater System - Black Diamond
JBL - 6" x 8" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black
Sony SRSHG1/BLK Hi-Res Wireless Speaker- Charcoal black
Verizon MiFi 6620L Jetpack 4G LTE Mobile Hotspot (Verizon Wireless)
CRX-322 CD Receiver
Slingbox M2
iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC)


In [0]:
filename = 'item,score.sav'
pickle.dump(User_item_score, open(filename, 'wb'))

In [0]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model(45880701, 14224)
print(result)

3.101815541770479


In [0]:
filename = 'user.csv'
pickle.dump(b, open(filename, 'wb'))

In [0]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model(45880701)
print(result)

TypeError: ignored

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(Test,final_movie))

ValueError: ignored